#Introduction

I started researching for this task. And got a amazing implementation online, from which i have adaopted this notebook.

Notebook Link: [https://colab.research.google.com/github/justinjohn0306/Wav2Lip/blob/master/Wav2Lip_simplified_v5.ipynb](https://colab.research.google.com/github/justinjohn0306/Wav2Lip/blob/master/Wav2Lip_simplified_v5.ipynb)

## Setting Up Environment

In [ ]:
from IPython.display import HTML, clear_output
import os
!rm -rf /content/sample_data
!mkdir /content/sample_data

!git clone https://github.com/justinjohn0306/Wav2Lip

%cd /content/Wav2Lip

#download the pretrained model
!wget 'https://github.com/justinjohn0306/Wav2Lip/releases/download/models/wav2lip.pth' -O 'checkpoints/wav2lip.pth'
!wget 'https://github.com/justinjohn0306/Wav2Lip/releases/download/models/wav2lip_gan.pth' -O 'checkpoints/wav2lip_gan.pth'
!wget 'https://github.com/justinjohn0306/Wav2Lip/releases/download/models/resnet50.pth' -O 'checkpoints/resnet50.pth'
!wget 'https://github.com/justinjohn0306/Wav2Lip/releases/download/models/mobilenet.pth' -O 'checkpoints/mobilenet.pth'
a = !pip install https://raw.githubusercontent.com/AwaleSajil/ghc/master/ghc-1.0-py3-none-any.whl
!pip install git+https://github.com/elliottzheng/batch-face.git@master

!pip install ffmpeg-python mediapipe==0.8.11

clear_output()
print("All set and ready!")

In [2]:
from IPython.display import HTML
from base64 import b64encode
def showVideo(path):
  mp4 = open(str(path),'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  return HTML("""
  <video width=700 controls>
        <source src="%s" type="video/mp4">
  </video>
  """ % data_url)

Below are some utility functions which split video and audio. This is to overcome the issue of the moving lips even when no face is present.

In [172]:
import ffmpeg
import soundfile

import moviepy.editor as mpe

def trim_video_and_save(input_file, output_file, start_time, end_time):
  video = mpe.VideoFileClip(input_file)
  split_video = video.subclip(start_time, end_time)
  split_video.write_videofile(output_file)

def trim_wav_audio_file(input_file, output_file, start_time, end_time):
  data, fs = soundfile.read(input_file)
  trimmed_data = data[int(start_time * fs):int(end_time * fs)]
  soundfile.write(output_file, trimmed_data, fs)

By maually watching the video, I have noted the time stamps for video with face and no face.   
For example,
>  0s to 8s - face is visible,  
>  8s to 22s - face is not visible  

so on.  
The below function splits the video and audio files

In [188]:
def split_video_and_audio(time_stamps):

  if not os.path.exists("/content/brokenfiles"):
    os.mkdir("/content/brokenfiles")

  face = True
  for time_stamp in range(len(time_stamps)-1):
    if face:
      trim_video_and_save("/content/youtube_video.mp4", f"/content/brokenfiles/video_face_{time_stamp}.mp4", time_stamps[time_stamp], time_stamps[time_stamp+1])
      trim_wav_audio_file("/content/output10.wav", f"/content/brokenfiles/audio_face_{time_stamp}.wav", time_stamps[time_stamp], time_stamps[time_stamp+1])
      face = False
    else:
      trim_video_and_save("/content/youtube_video.mp4", f"/content/brokenfiles/video_{time_stamp}.mp4", time_stamps[time_stamp], time_stamps[time_stamp+1])
      trim_wav_audio_file("/content/output10.wav", f"/content/brokenfiles/audio_{time_stamp}.wav", time_stamps[time_stamp], time_stamps[time_stamp+1])
      face = True
  return "success"

In [ ]:
time_stamps = [0,9,23,26,33,59]
split_video_and_audio(time_stamps)

In [190]:
%cd /content/Wav2Lip

/content/Wav2Lip


# Main Function  
below is the main function which syncs audio and video. we can any video or audio, we just have to upload it on google colab.

In [191]:
def lip_sync_audio_and_video(video_path, audio_path, output_file_path):

  if not os.path.exists("/content/results_1"):
      os.mkdir("/content/results_1")

  checkpoint_path = 'checkpoints/wav2lip.pth'

  pad_top =  0
  pad_bottom =  0
  pad_left =  0
  pad_right =  0
  rescaleFactor =  1
  nosmooth = False

  !python inference.py --checkpoint_path $checkpoint_path --face "$video_path" --audio "$audio_path" --outfile "$output_file_path" --pads $pad_top $pad_bottom $pad_left $pad_right --resize_factor $rescaleFactor

  # clear_output()
  print("audio and video synced!")

In [202]:
video_files_with_face = ["/content/brokenfiles/video_face_0.mp4", "/content/brokenfiles/video_face_2.mp4",
                         "/content/brokenfiles/video_face_4.mp4"]
audio_files_with_face = ["/content/brokenfiles/audio_face_0.wav", "/content/brokenfiles/audio_face_2.wav",
                         "/content/brokenfiles/audio_face_4.wav"]
video_files = ["/content/brokenfiles/video_1.mp4", "/content/brokenfiles/video_3.mp4"]
audio_files = ["/content/brokenfiles/audio_1.wav", "/content/brokenfiles/audio_3.wav"]

In [ ]:
for ind, fil in enumerate(video_files_with_face):
  lip_sync_audio_and_video(fil,audio_files_with_face[ind], f"/content/results_1/video_{ind}.mp4")

Now syncing the video with no face with audio

In [204]:
def attach_video_and_audio(video_file, audio_file, output_file):
  if not os.path.exists("/content/results_2"):
    os.mkdir("/content/results_2")

  video = mpe.VideoFileClip(video_file)
  audio = mpe.AudioFileClip(audio_file)
  joined_video = video.set_audio(audio)
  joined_video.write_videofile(output_file)

In [ ]:
for ind, fil in enumerate(video_files):
  attach_video_and_audio(fil,audio_files[ind], f"/content/results_2/video_{ind}.mp4")

Combining the videos

In [208]:
def attach_videos(video_files, output_file):

  video_clips = []
  for video_file in video_files:
    video_clips.append(mpe.VideoFileClip(video_file))

  concatenated_video = mpe.concatenate_videoclips(video_clips)
  concatenated_video.write_videofile(output_file)

In [ ]:
video_files = ["/content/results_1/video_0.mp4", "/content/results_2/video_0.mp4",
               "/content/results_1/video_1.mp4", "/content/results_2/video_1.mp4",
               "/content/results_1/video_2.mp4"]
output_file = "/content/final_video.mp4"

attach_videos(video_files, output_file)

In [210]:
showVideo("/content/final_video.mp4")